# TP1 : Visualisation et réduction de dimension sur MNIST (Corrigé)

Ce corrigé présente les solutions pour appliquer PCA, Isomap, Diffusion Maps et t-SNE sur MNIST.

## Objectifs et démarche
- Charger et explorer le jeu de données MNIST
- Appliquer différentes méthodes de réduction de dimension
- Visualiser et analyser les résultats

In [ ]:
# 1. Charger le jeu de données MNIST
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data / 255.0
y = mnist.target.astype(int)

In [ ]:
# 2. PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=y, cmap='tab10', s=2)
plt.title('PCA sur MNIST')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
# 3. Isomap
from sklearn.manifold import Isomap
isomap = Isomap(n_neighbors=10, n_components=2)
X_iso = isomap.fit_transform(X)
plt.figure(figsize=(8,6))
plt.scatter(X_iso[:,0], X_iso[:,1], c=y, cmap='tab10', s=2)
plt.title('Isomap sur MNIST')
plt.xlabel('Dim 1')
plt.ylabel('Dim 2')
plt.show()

In [ ]:
# 4. Diffusion Maps (via PyGSP)
# Nécessite l'installation de PyGSP : pip install pygsp
from pygsp import graphs, filters
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
import numpy.linalg as la

# Sous-échantillon pour la rapidité
X_sub = X[:1000]
y_sub = y[:1000]

# Construction du graphe k-NN
k = 10
knn = NearestNeighbors(n_neighbors=k)
knn.fit(X_sub)
distances, indices = knn.kneighbors(X_sub)

# Poids du graphe (gaussien)
sigma = np.mean(distances)
W = np.zeros((X_sub.shape[0], X_sub.shape[0]))
for i in range(X_sub.shape[0]):
    for j in indices[i]:
        W[i, j] = np.exp(-la.norm(X_sub[i] - X_sub[j])**2 / (2 * sigma**2))

# Normalisation du graphe
D = np.diag(W.sum(axis=1))
P = np.linalg.inv(D) @ W

# Calcul des valeurs propres/vecteurs propres
eigvals, eigvecs = la.eigh(P)
# On prend les deux plus grandes valeurs propres (hors la première)
X_diffmap = eigvecs[:, -2:]

# Visualisation
plt.figure(figsize=(8,6))
plt.scatter(X_diffmap[:,0], X_diffmap[:,1], c=y_sub, cmap='tab10', s=10)
plt.title('Diffusion Maps sur MNIST (sous-échantillon)')
plt.xlabel('DiffMap 1')
plt.ylabel('DiffMap 2')
plt.show()

In [ ]:
# 5. t-SNE
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X[:2000])  # t-SNE est coûteux, on prend un sous-échantillon
plt.figure(figsize=(8,6))
plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y[:2000], cmap='tab10', s=2)
plt.title('t-SNE sur MNIST')
plt.xlabel('Dim 1')
plt.ylabel('Dim 2')
plt.show()

## Analyse et comparaison
- **PCA** sépare globalement les chiffres mais les groupes se chevauchent, car la méthode est linéaire et ne capte pas la structure complexe des données.
- **Isomap** révèle mieux la structure intrinsèque en tenant compte de la géométrie du manifold, mais reste sensible au choix du nombre de voisins et au bruit.
- **Diffusion Maps** extrait des coordonnées non linéaires qui respectent la connectivité du graphe, ce qui permet de mieux séparer les classes et de révéler des structures globales. Cette méthode est robuste au bruit et à la connectivité locale, mais dépend du paramètre de voisinage et du noyau utilisé.
- **t-SNE** isole très bien les clusters de chiffres, en maximisant la séparation locale des classes. Il est particulièrement efficace pour la visualisation, mais la structure globale peut être déformée et les résultats varient selon les paramètres (perplexity, learning rate).

**Comparaison générale :**
- Les méthodes non linéaires (Isomap, Diffusion Maps, t-SNE) sont plus adaptées à la structure complexe de MNIST que PCA.
- t-SNE est le plus performant pour la visualisation des clusters, mais Diffusion Maps offre une meilleure robustesse et une interprétation plus globale.
- Le choix de la méthode dépend du but : visualisation locale (t-SNE), analyse globale (Diffusion Maps), ou réduction rapide (PCA).